In [1]:
# External libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from poliastro.bodies import Moon 
from scipy.linalg import block_diag
from filterpy.common import Saver
from astropy import units as u
from poliastro.twobody.propagation import cowell as cowell
from poliastro.twobody.propagation import farnocchia as farnocchia
from poliastro.twobody.propagation import vallado as vallado
from poliastro.twobody.propagation import mikkola as mikkola
from poliastro.twobody.propagation import markley as markley
from poliastro.twobody.propagation import pimienta as pimienta
from poliastro.twobody.propagation import gooding as gooding
from poliastro.twobody.propagation import danby as danby
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

In [2]:
############################################
dt = 10
############################################
df = pd.read_csv(f'DATA/ephemeris sat/inclination zero/{dt} step size.csv', header=3, sep=';') 
real_Latitudes, real_Longitudes, real_Altitudes = df['Lat (deg)'], df['Lon (deg)'], df['Alt (km)']
real_Vxs,real_Vys,real_Vzs = df['x (km/sec)'], df['y (km/sec)'],df['z (km/sec)']

real_X, real_Y, real_Z = [], [], []
for i in range(len(df)):
    altitude = real_Altitudes[i]
    latitude = real_Latitudes[i]
    longitude = real_Longitudes[i]
    x, y, z = spherical2cartesian(altitude, latitude, longitude)
    real_X.append(x)
    real_Y.append(y)
    real_Z.append(z)
real_X, real_Y, real_Z = np.array(real_X),np.array(real_Y),np.array(real_Z)

In [3]:
# Initial State Vector:
x, y, z = real_X[0], real_Y[0], real_Z[0]
vx, vy, vz = real_Vxs[0], real_Vys[0], real_Vzs[0]

In [4]:
FILE = pd.read_csv('Pos_Err_d100_quartile_40-60_tol1=5_tol2=0.9_t3=0.1_t4=0.1.csv')
H = FILE['H']
LAT = FILE['LAT']
LON = FILE['LON']
IDXs = FILE['ID']

In [5]:
def find_F_matrix(x:float,y:float,z:float)-> np.array:
    upleft = np.eye(3)
    bottleft = np.zeros((3,3))
    # bottleft = np.eye(3)
    bottright = np.eye(3)

    mi = 4.9048695e3 # km^3/s^2 	
    r = np.sqrt( x**2 + y**2 + z**2)


    J = np.zeros((3,3))
    # First Row
    J[0,0] = (3*mi*x**2/r**5) - mi/r**3
    J[0,1] = 3*mi*x*y/r**5
    J[0,2] = 3*mi*x*z/r**5 
    # Second Row
    J[1,0] = 3*mi*y*x/r**5
    J[1,1] = (3*mi*y**2/r**5) - mi/r**3 
    J[1,2] = 3*mi*y*z/r**5 
    # Third Row
    J[2,0] = 3*mi*z*x/r**5
    J[2,1] = 3*mi*z*y/r**5 
    J[2,2] = (3*mi*z**2/r**5) - mi/r**3 
    # End
    upright = J
    # Bulk
    tmp1 = np.hstack([upleft,upright])
    tmp2 = np.hstack([bottleft,bottright])
    phi = np.vstack([tmp1,tmp2])
    return phi

# Init

In [6]:
kf = KalmanFilter(dim_x = 6, dim_z = 3)
iX, iY, iZ, iVx, iVy, iVz  = 0,1,2,3,4,5

kf.F = find_F_matrix(x, y, z)

q1 = np.eye(3)
q1[0,0]  = 10  
q1[1,1]  = 30    
q1[2,2]  = 0.1


q2 = np.eye(3)
q2[0,0]=0.2 
q2[1,1]=0.2  
q2[2,2]=0.05


kf.Q = block_diag(q1, q2)
# Measurement Matrix
kf.H[iX, iX] = 1
kf.H[iZ, iZ] = 1
kf.H[iY, iY] = 1
kf.R *= 1  # KM 
kf.x = np.array([[x, y, z, vx, vy, vz]]).T
kf.u = 0

kf.P = np.eye(6) * 50

GATE_LIMIT = 3.0 # old 4.3
saver = Saver(kf)

# Predictor - Corrector

In [7]:
mus = []
Zs = []

# for idx in range(670):
for idx in range(678):
########################################################################
    # PREDICTOR:
    x,y,z = kf.x[0],kf.x[1],kf.x[2] 
    vx,vy,vz = kf.x[3],kf.x[4],kf.x[5]
    r = np.array([x,y,z])*u.km
    v = np.hstack([vx,vy,vz])*u.km/u.s
    tofs = np.linspace(10,10,1)*u.s                         # 10seconds
    # Propagate:
    rr, vv = cowell(Moon.k, r, v, tofs=tofs )
    # Calculate State Transition Matrix:
    foo  = rr.to_value()[0]
    foo2 = vv.to_value()[0]
    kf.F = find_F_matrix(foo[0],foo[1],foo[2])
    # Predict:
    kf.predict() # P
    kf.x = np.array([foo[0],foo[1],foo[2], foo2[0], foo2[1], foo2[2]  ])
########################################################################
    # CORRECTOR
    if H[idx] > 0:
        std_x = np.sqrt(kf.P[0,0])
        std_y = np.sqrt(kf.P[1,1])
        std_z = np.sqrt(kf.P[2,2])
        
        XX,YY,ZZ = spherical2cartesian(H[idx],LAT[idx],LON[idx]) # measurement
        Z = np.array([XX,YY,ZZ])
        
        resid = kf.residual_of(Z)

        if resid.shape != (3,):
            resid = np.array([resid[0,0], resid[1,1], resid[2,2]])
        


        if abs(resid[0]) > GATE_LIMIT * std_x or abs(resid[1]) > GATE_LIMIT * std_y or abs(resid[2]) > GATE_LIMIT * std_z:
            saver.save()
            Zs.append(Z)
            XX,YY,ZZ = 0, 0, 0  # reset
        else:
            print(f'UPDATE measurement:{idx}')
            kf.update(Z)
            saver.save()
            Zs.append(Z)
            XX,YY,ZZ = 0, 0, 0  # reset
    else:
        saver.save()
        Zs.append(np.array([np.NaN,np.NaN,np.NaN]))
    mus.append(kf.x)

UPDATE measurement:0
UPDATE measurement:1
UPDATE measurement:2
UPDATE measurement:3
UPDATE measurement:4
UPDATE measurement:5
UPDATE measurement:6
UPDATE measurement:8
UPDATE measurement:9
UPDATE measurement:11
UPDATE measurement:18
UPDATE measurement:21
UPDATE measurement:25
UPDATE measurement:27
UPDATE measurement:34
UPDATE measurement:36
UPDATE measurement:39
UPDATE measurement:42
UPDATE measurement:45
UPDATE measurement:47
UPDATE measurement:50
UPDATE measurement:53
UPDATE measurement:56
UPDATE measurement:59
UPDATE measurement:71
UPDATE measurement:74
UPDATE measurement:101
UPDATE measurement:105
UPDATE measurement:109
UPDATE measurement:112
UPDATE measurement:116
UPDATE measurement:124
UPDATE measurement:127
UPDATE measurement:142
UPDATE measurement:145
UPDATE measurement:148
UPDATE measurement:151
UPDATE measurement:154
UPDATE measurement:158
UPDATE measurement:223
UPDATE measurement:244
UPDATE measurement:245
UPDATE measurement:246
UPDATE measurement:248
UPDATE measurement:252


In [8]:
####################################################################
######################## - MEAS - ##################################
X_m = []
for i in range(len(Zs)):
    X_m.append(Zs[i][0])

Y_m = []
for i in range(len(Zs)):
    Y_m.append(Zs[i][1])

Z_m = []
for i in range(len(Zs)):
    Z_m.append(Zs[i][2])

In [9]:
X = saver.x
P = saver.P
P_arr = np.array(P)
Pxx, Pyy, Pzz = [], [], []
x_pred, y_pred, z_pred = [], [], []
for i in range(len(P)):
    Pxx.append(P[i][0,0])
    Pyy.append(P[i][1,1])
    Pzz.append(P[i][2,2])
    x_pred.append(X[i][0])
    y_pred.append(X[i][1])
    z_pred.append(X[i][2])
Pxx, Pyy, Pzz = np.array(Pxx), np.array(Pyy), np.array(Pzz)
Pxx, Pyy, Pzz = np.sqrt(Pxx), np.sqrt(Pyy), np.sqrt(Pzz)

x_true = real_X[:len(x_pred)]
y_true = real_Y[:len(y_pred)]
z_true = real_Z[:len(z_pred)]

# Plot Figure

In [10]:
lw = 1
S =3

plt.figure(dpi=250, tight_layout=True)
# plt.plot(Pxx, '--r', linewidt5=lw)
# plt.plot(-Pxx, '--r', linewidth=lw)
plt.plot(x_pred, '-k', linewidth=lw)
plt.plot(x_true, 'r', linewidth=lw)
plt.scatter(IDXs,X_m, s=S)
plt.legend(['Position estimated by KF','Real position','measurements'])
plt.xlabel(f'Step Size: {dt}')
# plt.xlim([0, len(x_true)//2])
# plt.ylim([-550, 550])
plt.ylabel('X [Km]')
plt.savefig('fig-A1.jpg')
plt.show()

plt.figure(dpi=250, tight_layout=True)
# plt.plot(Pxx, '--r', linewidth=lw)
# plt.plot(-Pxx, '--r', linewidth=lw)
plt.plot(y_pred, '-k', linewidth=lw)
plt.plot(y_true, 'r', linewidth=lw)
plt.scatter(IDXs,Y_m, s=S)
plt.legend(['Position estimated by KF','Real position','measurements'])
plt.xlabel(f'Step Size: {dt}')
# plt.xlim([0, len(x_true)//2])
plt.ylabel('Y [Km]')
plt.savefig('fig-A2.jpg')
plt.show()

plt.figure(dpi=250, tight_layout=True)
# plt.plot(Pxx, '--r', linewidth=lw)
# plt.plot(-Pxx, '--r', linewidth=lw)
plt.plot(z_pred, '-k', linewidth=lw)
plt.plot(z_true, 'r', linewidth=lw)
plt.scatter(IDXs,Z_m, s=S)
plt.legend(['Position estimated by KF','Real position','measurements'])
plt.xlabel(f'Step Size: {dt}')
# plt.xlim([0, len(x_true)//2])
plt.ylabel('Z [Km]')
plt.savefig('fig-A3.jpg')
plt.show()

# Errors 

In [11]:
plt.figure(dpi=180, tight_layout=True, figsize=(10,10))
lw = 0.8

plt.subplot(311)
x_pred = np.array(x_pred)
x_true = np.array(x_true)
diff_x = []
for x,y in zip(x_pred,x_true):
    d = (x - y)
    diff_x.append(d)
plt.title('Error along X (LCLF)')
plt.plot(diff_x, '-k', linewidth=lw)
# plt.ylim([-5,5])
plt.xlim([0,670])
plt.ylabel('Km')
plt.show()

plt.subplot(312)
y_pred = np.array(y_pred)
y_true = np.array(y_true)
diff_y = []
for x,y in zip(y_pred,y_true):
    d = (x - y)
    diff_y.append(d)
plt.title('Error along Y (LCLF)')
# plt.plot(Pyy, '--r', linewidth=lw)
plt.plot(diff_y, '-k', linewidth=lw)
plt.xlim([0,670])
# plt.ylim([-5,5])
plt.ylabel('Km')
plt.show()

plt.subplot(313)
z_pred = np.array(z_pred)
z_true = np.array(z_true)
diff_z = []
for x,y in zip(z_pred,z_true):
    d = (x - y)
    diff_z.append(d)
plt.title('Error along Z (LCLF)')
plt.xlim([0,670])
plt.ylim([-1,1])
# plt.plot(Pzz, '--r', linewidth=lw)
plt.plot(diff_z, '-k', linewidth=lw)
plt.xlabel(f'Step Size: {dt}')
plt.ylabel('Km')
plt.savefig('fig-B.jpg')
plt.show()

In [12]:
plt.figure(dpi=170, tight_layout=True, figsize=(13,13))
lw = 0.8

plt.subplot(311)
x_pred = np.array(x_pred)
x_true = np.array(x_true)
diff = []
for x,y in zip(x_pred,x_true):
    d = (x - y)
    diff.append(d)
plt.title('Error along X (LCLF)')
plt.plot(3*Pxx, '--r', linewidth=lw)
plt.plot(-3*Pxx, '--r', linewidth=lw)
plt.plot(diff, '-k', linewidth=lw)
plt.legend(['$\pm 3\sigma$','_nolegend_','error on KF estimation'], loc='upper right')
plt.ylim([-30,30])
plt.xlim([0,670])
plt.ylabel('Km')
plt.show()

plt.subplot(312)
y_pred = np.array(y_pred)
y_true = np.array(y_true)
diff = []
for x,y in zip(y_pred,y_true):
    d = (x - y)
    diff.append(d)
plt.title('Error along Y (LCLF)')
plt.plot(3*Pyy, '--r', linewidth=lw)
plt.plot(-3*Pyy, '--r', linewidth=lw)
plt.plot(diff, '-k', linewidth=lw)
plt.xlim([0,670])
plt.ylim([-30,30])
plt.legend(['$\pm 3\sigma$','_nolegend_','error on KF estimation'], loc='upper right')
plt.ylabel('Km')
plt.show()

plt.subplot(313)
z_pred = np.array(z_pred)
z_true = np.array(z_true)
diff = []
for x,y in zip(z_pred,z_true):
    d = (x - y)
    diff.append(d)
plt.title('Error along Z (LCLF)')
plt.xlim([0,670])
plt.ylim([-10,10])
plt.plot(3*Pzz, '--r', linewidth=lw)
plt.plot(-3*Pzz, '--r', linewidth=lw)
plt.plot(diff, '-k', linewidth=lw)
plt.legend(['$\pm 3\sigma$','_nolegend_','error on KF estimation'], loc='upper right')
plt.xlabel(f'Step Size: {dt}')
plt.ylabel('Km')
plt.savefig('fig-C.jpg')

plt.show()

In [13]:
plt.figure(dpi=200, tight_layout=True, figsize=(10,10))
lw = 0.8

plt.subplot(211)
x_pred = np.array(x_pred)
x_true = np.array(x_true)
diff = []
for x,y in zip(x_pred,x_true):
    d = (x - y)
    diff.append(d)
plt.title('Error along X (LCLF)')
plt.plot(3*Pxx, '--r', linewidth=lw)
plt.plot(-3*Pxx, '--r', linewidth=lw)
plt.plot(diff, '-k', linewidth=lw)
plt.legend(['$\pm 3\sigma$','_nolegend_','error on KF estimation'], loc='upper right')
plt.xlabel(f'Step Size: {dt}')
plt.ylim([-25,25])
plt.xlim([0,670])
plt.ylabel('Km')

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, : ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,1*int(Img.shape[1]/10),2*int(Img.shape[1]/10),3*int(Img.shape[1]/10),4*int(Img.shape[1]/10),5*int(Img.shape[1]/10),6*int(Img.shape[1]/10),7*int(Img.shape[1]/10),8*int(Img.shape[1]/10),9*int(Img.shape[1]/10),Img.shape[1]], ['-147°',f'{-147+36*1}°',f'{-147+36*2}°',f'{-147+36*3}°',f'{-147+36*4}°',f'{-147+36*5}°',f'{-147+36*6}°',f'{-147+36*7}°',f'{-147+36*8}°',f'{-147+36*9}°','-147°']    )
plt.xlabel('LON')
plt.ylabel('LAT')
plt.savefig('fig-D1.jpg')
plt.show()

plt.figure(dpi=200, tight_layout=True, figsize=(10,10))
plt.subplot(211)
y_pred = np.array(y_pred)
y_true = np.array(y_true)
diff = []
for x,y in zip(y_pred,y_true):
    d = (x - y)
    diff.append(d)
plt.title('Error along Y (LCLF)')
plt.plot(3*Pyy, '--r', linewidth=lw)
plt.plot(-3*Pyy, '--r', linewidth=lw)
plt.plot(diff, '-k', linewidth=lw)
plt.xlim([0,670])
plt.ylim([-25,25])
plt.xlabel(f'Step Size: {dt}')
plt.legend(['$\pm 3\sigma$','_nolegend_','error on KF estimation'], loc='upper right')
plt.ylabel('Km')

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, : ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,1*int(Img.shape[1]/10),2*int(Img.shape[1]/10),3*int(Img.shape[1]/10),4*int(Img.shape[1]/10),5*int(Img.shape[1]/10),6*int(Img.shape[1]/10),7*int(Img.shape[1]/10),8*int(Img.shape[1]/10),9*int(Img.shape[1]/10),Img.shape[1]], ['-147°',f'{-147+36*1}°',f'{-147+36*2}°',f'{-147+36*3}°',f'{-147+36*4}°',f'{-147+36*5}°',f'{-147+36*6}°',f'{-147+36*7}°',f'{-147+36*8}°',f'{-147+36*9}°','-147°']    )
plt.xlabel('LON')
plt.ylabel('LAT')
plt.savefig('fig-D2.jpg')
plt.show()

plt.figure(dpi=200, tight_layout=True, figsize=(10,10))
plt.subplot(211)
z_pred = np.array(z_pred)
z_true = np.array(z_true)
diff = []
for x,y in zip(z_pred,z_true):
    d = (x - y)
    diff.append(d)
plt.title('Error along Z (LCLF)')
plt.xlim([0,670])
plt.ylim([-5,5])
plt.plot(3*Pzz, '--r', linewidth=lw)
plt.plot(-3*Pzz, '--r', linewidth=lw)
plt.plot(diff, '-k', linewidth=lw)
plt.xlabel(f'Step Size: {dt}')
plt.legend(['$\pm 3\sigma$','_nolegend_','error on KF estimation'], loc='upper right')
plt.xlabel(f'Step Size: {dt}')
plt.ylabel('Km')

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, : ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,1*int(Img.shape[1]/10),2*int(Img.shape[1]/10),3*int(Img.shape[1]/10),4*int(Img.shape[1]/10),5*int(Img.shape[1]/10),6*int(Img.shape[1]/10),7*int(Img.shape[1]/10),8*int(Img.shape[1]/10),9*int(Img.shape[1]/10),Img.shape[1]], ['-147°',f'{-147+36*1}°',f'{-147+36*2}°',f'{-147+36*3}°',f'{-147+36*4}°',f'{-147+36*5}°',f'{-147+36*6}°',f'{-147+36*7}°',f'{-147+36*8}°',f'{-147+36*9}°','-147°']    )
plt.xlabel('LON')
plt.ylabel('LAT')
plt.savefig('fig-D3.jpg')

plt.show()

# Error On Local Plane

In [14]:
idx = 0
E,N,U = [],[],[]
for x,y,z in zip(x_pred,y_pred,z_pred):
    x,y,z = x*1000, y*1000, z*1000
    xEast, yNorth, zUp = ecef_to_enu(x, y, z, real_Latitudes[idx], real_Longitudes[idx], real_Altitudes[idx]*1000)
    idx+=1
    E.append(xEast)
    N.append(yNorth)
    U.append(zUp)

E = np.array(E)/1000
N = np.array(N)/1000
U = np.array(U)/1000

In [15]:
plt.figure(dpi=220, figsize=(10,5))
plt.subplot(211)
plt.plot(E,'-k', linewidth=1)
plt.xlim([0,679])
plt.legend(['East'])
plt.title('Local error on position')
plt.ylabel('Km')
plt.xlabel('Step size: 10')

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, : ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,1*int(Img.shape[1]/10),2*int(Img.shape[1]/10),3*int(Img.shape[1]/10),4*int(Img.shape[1]/10),5*int(Img.shape[1]/10),6*int(Img.shape[1]/10),7*int(Img.shape[1]/10),8*int(Img.shape[1]/10),9*int(Img.shape[1]/10),Img.shape[1]], ['-147°',f'{-147+36*1}°',f'{-147+36*2}°',f'{-147+36*3}°',f'{-147+36*4}°',f'{-147+36*5}°',f'{-147+36*6}°',f'{-147+36*7}°',f'{-147+36*8}°',f'{-147+36*9}°','-147°']    )
plt.savefig('E1.jpg')
plt.xlabel('LON')
plt.ylabel('LAT')
plt.show()

plt.figure(dpi=220, figsize=(10,5))
plt.subplot(211)
plt.plot(N,'-k', linewidth=1)
plt.xlim([0,679])
plt.title('Local error on position')
plt.xlabel('Step size: 10')
plt.ylabel('Km')
plt.legend(['North'])
plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, : ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,1*int(Img.shape[1]/10),2*int(Img.shape[1]/10),3*int(Img.shape[1]/10),4*int(Img.shape[1]/10),5*int(Img.shape[1]/10),6*int(Img.shape[1]/10),7*int(Img.shape[1]/10),8*int(Img.shape[1]/10),9*int(Img.shape[1]/10),Img.shape[1]], ['-147°',f'{-147+36*1}°',f'{-147+36*2}°',f'{-147+36*3}°',f'{-147+36*4}°',f'{-147+36*5}°',f'{-147+36*6}°',f'{-147+36*7}°',f'{-147+36*8}°',f'{-147+36*9}°','-147°']    )
plt.xlabel('LON')
plt.ylabel('LAT')
plt.savefig('E2.jpg')
plt.show()

plt.figure(dpi=220, figsize=(10,5))
plt.subplot(211)
plt.plot(U,'-k', linewidth=1)
plt.xlim([0,679])
plt.xlabel('Step size: 10')
plt.ylabel('Km')
plt.legend(['Up'])
plt.title('Local error on position')
plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, : ])
plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
plt.xticks([0,1*int(Img.shape[1]/10),2*int(Img.shape[1]/10),3*int(Img.shape[1]/10),4*int(Img.shape[1]/10),5*int(Img.shape[1]/10),6*int(Img.shape[1]/10),7*int(Img.shape[1]/10),8*int(Img.shape[1]/10),9*int(Img.shape[1]/10),Img.shape[1]], ['-147°',f'{-147+36*1}°',f'{-147+36*2}°',f'{-147+36*3}°',f'{-147+36*4}°',f'{-147+36*5}°',f'{-147+36*6}°',f'{-147+36*7}°',f'{-147+36*8}°',f'{-147+36*9}°','-147°']    )
plt.xlabel('LON')
plt.ylabel('LAT')
plt.savefig('E3.jpg')
plt.show()

In [16]:
plt.figure(dpi=250)
plt.hist(E, bins=300)
plt.legend(['East'])
plt.title('Histogram of error in local frame')
plt.xlabel('Km')
plt.xlim([-3,3])
plt.savefig('F1.jpg')
plt.show()

plt.figure(dpi=250)
plt.hist(N, bins=100)
plt.legend(['North'])
plt.xlim([-0.7,0.7])
plt.title('Histogram of error in local frame')
plt.xlabel('Km')
plt.savefig('F2.jpg')
plt.show()

plt.figure(dpi=250)
plt.hist(U, bins=200)
plt.legend(['Up'])
plt.xlim([-3,3])
plt.title('Histogram of error in local frame')
plt.xlabel('Km')
plt.savefig('F3.jpg')
plt.show()

In [17]:
print('Zona Buona:')
print('MEAN')
print(f'East:{np.mean(E[500:]):.2f}, North:{np.mean(N[500:]):.2f}, Up:{np.mean(U[500:]):.2f}')
print('STD')
print(f'East:{np.std(E[500:]):.2f}, North:{np.std(N[500:]):.2f}, Up:{np.std(U[500:]):.2f}')

print('Zona Cattiva:')
print('MEAN')
print(f'East:{np.mean(E[150:350]):.2f}, North:{np.mean(N[150:350]):.2f}, Up:{np.mean(U[150:350]):.2f}')
print('STD')
print(f'East:{np.std(E[150:350]):.2f}, North:{np.std(N[150:350]):.2f}, Up:{np.std(U[150:350]):.2f}')

print('Intera orbita:')
print('MEAN')
print(f'East:{np.mean(E[:]):.2f}, North:{np.mean(N[:]):.2f}, Up:{np.mean(U[:]):.2f}')
print('STD')
print(f'East:{np.std(E[:]):.2f}, North:{np.std(N[:]):.2f}, Up:{np.std(U[:]):.2f}')

Zona Buona:
MEAN
East:-0.25, North:0.10, Up:0.20
STD
East:0.26, North:0.14, Up:0.42
Zona Cattiva:
MEAN
East:2.90, North:-0.07, Up:0.39
STD
East:4.04, North:0.26, Up:0.85
Intera orbita:
MEAN
East:0.95, North:0.03, Up:0.21
STD
East:2.63, North:0.22, Up:1.13


# Plotting: X coordinate - LCLC frame

In [25]:
# lw = 1
# S =3

# plt.figure(dpi=250, tight_layout=True)
# plt.subplot(211)
# plt.plot(x_true, 'r', linewidth=lw)
# plt.plot(Pxx+x_true, '--r', linewidth=lw)
# plt.plot(-Pxx+x_true, '--r', linewidth=lw)
# plt.plot(x_pred, '-k', linewidth=lw)
# plt.scatter(range(len(X_m)),X_m, s=S)
# plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
# plt.xlabel(f'Step Size: {dt}')
# plt.xlim([0, len(x_true)//2])
# plt.ylabel('X [Km]')
# plt.show()

# plt.subplot(212)
# Img = cv2.imread('4.bmp')
# plt.imshow(Img[235:335, :Img.shape[1]//2 ])
# plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
# plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
# plt.xlabel('LON')
# plt.ylabel('LAT')
# plt.savefig('AA1.jpg')
# plt.show()

# plt.figure(dpi=250, tight_layout=True)
# plt.subplot(211)
# plt.plot(x_true, 'r', linewidth=lw)
# plt.plot( Pxx+x_true, '--r', linewidth=lw)
# plt.plot(-Pxx+x_true, '--r', linewidth=lw)
# plt.plot(x_pred, '-k', linewidth=lw)
# plt.scatter(range(len(X_m)),X_m, s=S)
# plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
# plt.xlabel(f'Step Size: {dt}')
# plt.xlim([len(x_true)//2, len(x_true)])
# plt.ylabel('X [Km]')
# plt.show()

# plt.subplot(212)
# Img = cv2.imread('4.bmp')
# plt.imshow(Img[235:335, Img.shape[1]//2:Img.shape[1] ])
# plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
# plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])

# plt.xlabel('LON')
# plt.ylabel('LAT')
# plt.savefig('AA2.jpg')
# plt.show()

# Y LCLF

In [26]:
# plt.figure(dpi=250, tight_layout=True)
# plt.subplot(211)
# plt.plot(y_true, 'r', linewidth=lw)
# plt.plot(Pyy+y_true, '--r', linewidth=lw)
# plt.plot(-Pyy+y_true, '--r', linewidth=lw)
# plt.plot(y_pred, '-k', linewidth=lw)
# plt.scatter(range(len(Y_m)),Y_m, s=S)
# plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
# plt.xlabel(f'Step Size: {dt}')
# plt.xlim([0, len(y_true)//2])
# # plt.ylim([-5e7,5e7])
# plt.ylabel('Y [Km]')
# plt.show()

# plt.subplot(212)
# Img = cv2.imread('4.bmp')
# plt.imshow(Img[235:335, :Img.shape[1]//2 ])
# plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
# plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
# plt.xlabel('LON')
# plt.ylabel('LAT')
# plt.savefig('BB1.jpg')
# plt.show()



# plt.figure(dpi=250, tight_layout=True)
# plt.subplot(211)
# plt.plot(y_true, 'r', linewidth=lw)
# plt.plot( Pyy+y_true, '--r', linewidth=lw)
# plt.plot(-Pyy+y_true, '--r', linewidth=lw)
# plt.plot(y_pred, '-k', linewidth=lw)
# plt.scatter(range(len(Y_m)),Y_m, s=S)
# plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
# plt.xlabel(f'Step Size: {dt}')
# plt.xlim([len(y_true)//2, len(y_true)])
# plt.ylabel('Y [Km]')

# plt.subplot(212)
# Img = cv2.imread('4.bmp')
# plt.imshow(Img[235:335, Img.shape[1]//2:Img.shape[1] ])
# plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
# plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])

# plt.xlabel('LON')
# plt.ylabel('LAT')
# plt.savefig('BB22.jpg')
# plt.show()

# Z LCLF

In [27]:
# plt.figure(dpi=250, tight_layout=True)
# plt.subplot(211)
# plt.plot(z_true, 'r', linewidth=lw)
# plt.plot(Pzz+z_true, '--r', linewidth=lw)
# plt.plot(-Pzz+z_true, '--r', linewidth=lw)
# plt.plot(z_pred, '-k', linewidth=lw)
# plt.scatter(range(len(Z_m)),Z_m, s=S)
# plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
# plt.xlabel(f'Step Size: {dt}')
# plt.xlim([0, len(z_true)//2])
# plt.ylabel('Z [Km]')
# plt.show()

# plt.subplot(212)
# Img = cv2.imread('4.bmp')
# plt.imshow(Img[235:335, :Img.shape[1]//2 ])
# plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
# plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
# plt.xlabel('LON')
# plt.ylabel('LAT')
# plt.savefig('CC1.jpg')
# plt.show()



# plt.figure(dpi=250, tight_layout=True)
# plt.subplot(211)
# plt.plot(z_true, 'r', linewidth=lw)
# plt.plot( Pzz+z_true, '--r', linewidth=lw)
# plt.plot(-Pzz+z_true, '--r', linewidth=lw)
# plt.plot(z_pred, '-k', linewidth=lw)
# plt.scatter(range(len(Z_m)),Z_m, s=S)
# plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
# plt.xlabel(f'Step Size: {dt}')
# plt.xlim([len(z_true)//2, len(z_true)])
# plt.ylabel('Z [Km]')
# plt.show()

# plt.subplot(212)
# Img = cv2.imread('4.bmp')
# plt.imshow(Img[235:335, Img.shape[1]//2:Img.shape[1] ])
# plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
# plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])

# plt.xlabel('LON')
# plt.ylabel('LAT')
# plt.savefig('CC2.jpg')
# plt.show()

# Velocities

In [36]:
# X = saver.x
# X_arr = np.array(X)
# Vxx, Vyy, Vzz = [], [], []
# for i in range(len(X)):
#     Vxx.append(X[i][3])
#     Vyy.append(X[i][4])
#     Vzz.append(X[i][5])
# Vxx, Vyy, Vzz = np.array(Vxx), np.array(Vyy), np.array(Vzz)


# plt.figure(dpi=250, tight_layout=True)
# plt.subplot(211)
# plt.plot(real_Vxs, 'r', linewidth=lw)
# # plt.plot(Pzz+Vxx, '--r', linewidth=lw)
# # plt.plot(-Pzz+Vxx, '--r', linewidth=lw)
# plt.plot(Vxx, '-k', linewidth=lw)
# # plt.legend(['True Velocity','$\pm\sigma$','_nolegend_','Velocity estimated by KF'])
# plt.legend(['True Velocity','Velocity estimated by KF'])
# plt.xlabel(f'Step Size: {dt}')
# plt.xlim([0, len(real_Vxs)//2])
# plt.ylabel('Vx [Km/s]')
# plt.show()

# plt.subplot(212)
# Img = cv2.imread('4.bmp')
# plt.imshow(Img[235:335, :Img.shape[1]//2 ])
# plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
# plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
# plt.xlabel('LON')
# plt.ylabel('LAT')
# plt.savefig('DD1.jpg')
# plt.show()



# plt.figure(dpi=250, tight_layout=True)
# plt.subplot(211)
# plt.plot(real_Vxs, 'r', linewidth=lw)
# # plt.plot( Pzz+z_true, '--r', linewidth=lw)
# # plt.plot(-Pzz+z_true, '--r', linewidth=lw)
# plt.plot(Vxx, '-k', linewidth=lw)
# plt.legend(['True Velocity','Velocity estimated by KF'])
# # plt.legend(['True Velocity','$\pm\sigma$','_nolegend_','Velocity estimated by KF'])
# plt.xlabel(f'Step Size: {dt}')
# plt.xlim([len(Vxx)//2, len(Vxx)])
# plt.ylabel('Vx [Km/s]')
# plt.show()

# plt.subplot(212)
# Img = cv2.imread('4.bmp')
# plt.imshow(Img[235:335, Img.shape[1]//2:Img.shape[1] ])
# plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
# plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])

# plt.xlabel('LON')
# plt.ylabel('LAT')
# plt.savefig('DD2.jpg')
# plt.show()

In [32]:
# plt.figure(dpi=250, tight_layout=True)
# plt.subplot(211)
# plt.plot(real_Vys, 'r', linewidth=lw)
# # plt.plot(Pzz+Vxx, '--r', linewidth=lw)
# # plt.plot(-Pzz+Vxx, '--r', linewidth=lw)
# plt.plot(Vyy, '-k', linewidth=lw)
# # plt.legend(['True Velocity','$\pm\sigma$','_nolegend_','Velocity estimated by KF'])
# plt.legend(['True Velocity','Velocity estimated by KF'])
# plt.xlabel(f'Step Size: {dt}')
# plt.xlim([0, len(real_Vys)//2])
# plt.ylabel('Vy [Km/s]')
# plt.show()

# plt.subplot(212)
# Img = cv2.imread('4.bmp')
# plt.imshow(Img[235:335, :Img.shape[1]//2 ])
# plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
# plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
# plt.xlabel('LON')
# plt.ylabel('LAT')
# plt.savefig('EE1.jpg')

# plt.show()



# plt.figure(dpi=250, tight_layout=True)
# plt.subplot(211)
# plt.plot(real_Vys, 'r', linewidth=lw)
# # plt.plot( Pzz+z_true, '--r', linewidth=lw)
# # plt.plot(-Pzz+z_true, '--r', linewidth=lw)
# plt.plot(Vyy, '-k', linewidth=lw)
# plt.legend(['True Velocity','Velocity estimated by KF'])
# # plt.legend(['True Velocity','$\pm\sigma$','_nolegend_','Velocity estimated by KF'])
# plt.xlabel(f'Step Size: {dt}')
# plt.xlim([len(Vyy)//2, len(Vyy)])
# plt.ylabel('Vy [Km/s]')
# plt.show()

# plt.subplot(212)
# Img = cv2.imread('4.bmp')
# plt.imshow(Img[235:335, Img.shape[1]//2:Img.shape[1] ])
# plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
# plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])

# plt.xlabel('LON')
# plt.ylabel('LAT')
# plt.savefig('EE2.jpg')

# plt.show()

In [37]:
# plt.figure(dpi=250, tight_layout=True)
# plt.subplot(211)
# plt.plot(real_Vzs, 'r', linewidth=lw)
# # plt.plot(Pzz+Vxx, '--r', linewidth=lw)
# # plt.plot(-Pzz+Vxx, '--r', linewidth=lw)
# Vzz = np.array(Vzz)*1000
# plt.plot(Vzz, '-k', linewidth=lw)
# # plt.legend(['True Velocity','$\pm\sigma$','_nolegend_','Velocity estimated by KF'])
# plt.legend(['True Velocity','Velocity estimated by KF'])
# plt.xlabel(f'Step Size: {dt}')
# plt.xlim([0, len(real_Vys)//2])
# plt.ylabel('Vz [m/s]')
# plt.show()

# plt.subplot(212)
# Img = cv2.imread('4.bmp')
# plt.imshow(Img[235:335, :Img.shape[1]//2 ])
# plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
# plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'-147°',f'{-147+18}°',f'{-147+2*18}°',f'{-147+3*18}°',f'{-147+4*18}°',f'{-147+5*18}°',f'{-147+6*18}°',f'{-147+7*18}°',f'{-147+8*18}°',f'{-147+9*18}°',f'{-147+18*10}°'])
# plt.xlabel('LON')
# plt.ylabel('LAT')
# plt.savefig('FF1.jpg')
# plt.show()



# plt.figure(dpi=250, tight_layout=True)
# plt.subplot(211)
# plt.plot(real_Vzs, 'r', linewidth=lw)
# # plt.plot( Pzz+z_true, '--r', linewidth=lw)
# # plt.plot(-Pzz+z_true, '--r', linewidth=lw)
# plt.plot(Vzz, '-k', linewidth=lw)
# plt.legend(['True Velocity','Velocity estimated by KF'])
# # plt.legend(['True Velocity','$\pm\sigma$','_nolegend_','Velocity estimated by KF'])
# plt.xlabel(f'Step Size: {dt}')
# plt.xlim([len(Vzz)//2, len(Vzz)])
# plt.ylabel('Vz [m/s]')
# plt.show()

# plt.subplot(212)
# Img = cv2.imread('4.bmp')
# plt.imshow(Img[235:335, Img.shape[1]//2:Img.shape[1] ])
# plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
# plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])

# plt.xlabel('LON')
# plt.ylabel('LAT')
# plt.savefig('FF2.jpg')
# plt.show()